In [ ]:
# Detectron2 basic setup
from detectron2.engine import DefaultTrainer, DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.logger import setup_logger
setup_logger()

# Common libraries
import numpy as np
import os, json, cv2, random

# Detectron2 utilities
from detectron2 import model_zoo
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.structures import BoxMode

In [ ]:
import numpy as np
import cv2
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [ ]:
# Detectron2 engine and visualization
from detectron2.engine import DefaultTrainer, DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.logger import setup_logger
setup_logger()

from detectron2 import model_zoo
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.structures import BoxMode

# General utilities
import numpy as np
import os, json, cv2, random

In [ ]:
from detectron2.data.datasets import register_coco_instances

# Register training dataset
register_coco_instances(
    "ddh_train",
    {},
    "path/to/your/train_annotations",  # Set the path to your COCO annotations JSON for training
    "path/to/your/train_images"        # Set the path to the image root directory for the training split
)

# Register validation dataset
register_coco_instances(
    "ddh_val",
    {},
    "path/to/your/val_annotations",    # Set the path to your COCO annotations JSON for validation
    "path/to/your/val_images"          # Set the path to the image root directory for the validation split
)

In [ ]:
from detectron2.data import DatasetCatalog
dataset_dicts = DatasetCatalog.get("ddh_val")
print(dataset_dicts[0].keys())  # 'annotations' or 'instances' があるか確認

In [ ]:

from detectron2.data import MetadataCatalog

# Define keypoint names, left-right pairs, and skeleton connections
keypoint_names = ['rtacetab1', 'rtacetab2', 'rtilleum', 'rtfemur1', 'rtfemur2',
                  'ltacetab1', 'ltacetab2', 'ltilleum', 'ltfemur1', 'ltfemur2']

keypoint_flip_map = [
    ('rtacetab1', 'ltacetab1'),
    ('rtilleum', 'ltilleum'),
    ('rtacetab2', 'ltacetab2'),
    ('rtfemur1', 'ltfemur1'),
    ('rtfemur2', 'ltfemur2')
]

keypoint_connection_rules = [
    ('rtacetab1', 'rtacetab2', (0, 255, 255)),
    ('rtfemur1', 'rtfemur2', (0, 100, 100)),
    ('rtilleum', 'ltilleum', (0, 255, 0)),
    ('ltacetab1', 'ltacetab2', (255, 255, 0)),
    ('ltfemur1', 'ltfemur2', (255, 0, 255))
]

# Match class name to COCO JSON categories[0]['name'] (e.g., 'DDH')
MetadataCatalog.get("ddh_train").thing_classes = ["DDH"]
MetadataCatalog.get("ddh_train").keypoint_names = keypoint_names
MetadataCatalog.get("ddh_train").keypoint_flip_map = keypoint_flip_map
MetadataCatalog.get("ddh_train").keypoint_connection_rules = keypoint_connection_rules

# Apply the same settings to 'ddh_val'
MetadataCatalog.get("ddh_val").thing_classes = ["DDH"]
MetadataCatalog.get("ddh_val").keypoint_names = keypoint_names
MetadataCatalog.get("ddh_val").keypoint_flip_map = keypoint_flip_map
MetadataCatalog.get("ddh_val").keypoint_connection_rules = keypoint_connection_rules

# Check metadata
metadata = MetadataCatalog.get("ddh_train")
print(metadata.as_dict())

In [ ]:
train_dataset_dicts = DatasetCatalog.get("ddh_train")
val_dataset_dicts = DatasetCatalog.get("ddh_val")

# Quick sanity check of counts (example)
print(f"Train images: {len(train_dataset_dicts)}")
print(f"Validation images: {len(val_dataset_dicts)}")

In [ ]:
metadata = MetadataCatalog.get("ddh_train")
metadata.as_dict()

In [ ]:
import random
from detectron2.utils.visualizer import Visualizer

metadata = MetadataCatalog.get("ddh_train")
dataset_dicts = DatasetCatalog.get("ddh_train")
def cv2_imshow(im):
    im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    plt.figure(), plt.imshow(im), plt.axis('off');

for d in random.sample(dataset_dicts, 5):
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=metadata, scale=1.0)   
    vis = visualizer.draw_dataset_dict(d)
    cv2_imshow(vis.get_image()[:, :, ::-1])

In [ ]:
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
from detectron2 import model_zoo
import os

cfg = get_cfg()
cfg.MODEL.DEVICE = "cuda"

# Model to use (ResNet-50 + FPN, pretrained on COCO keypoints)
cfg.merge_from_file(model_zoo.get_config_file("COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x.yaml"))

# Dataset names (as registered by register_coco_instances)
cfg.DATASETS.TRAIN = ("ddh_train",)
cfg.DATASETS.TEST = ("ddh_val",)

# Output directory (for checkpoints and logs)
cfg.OUTPUT_DIR = "path/to/your/output_dir"  # Set the directory to save checkpoints and logs
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

# Evaluation period (in iterations)
cfg.TEST.EVAL_PERIOD = 500  # e.g., evaluate on the validation set every 500 iterations

# Other training settings (batch size, etc.)
cfg.SOLVER.IMS_PER_BATCH = 4
cfg.SOLVER.BASE_LR = 0.001
cfg.SOLVER.MAX_ITER = 35000
cfg.SOLVER.STEPS = []  # no step LR decay

# Number of keypoints (10)
cfg.MODEL.ROI_KEYPOINT_HEAD.NUM_KEYPOINTS = 10
cfg.TEST.KEYPOINT_OKS_SIGMAS = [0.05] * 10

In [ ]:
from detectron2.engine import DefaultTrainer
from detectron2.evaluation import COCOEvaluator

class MyTrainer(DefaultTrainer):
    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        if output_folder is None:
            output_folder = os.path.join(cfg.OUTPUT_DIR, "inference")
        return COCOEvaluator(dataset_name, cfg, False, output_folder)

In [ ]:
from detectron2.engine.hooks import HookBase
import os

class BestCheckpointer(HookBase):
    def __init__(self, eval_period, checkpointer, val_metric_name="val_loss", mode="min", file_prefix="model_best"):
        self._period = eval_period
        self._checkpointer = checkpointer
        self._val_metric = val_metric_name
        self._mode = mode
        self._file_prefix = file_prefix
        self._best_metric = None

    def after_step(self):
        next_iter = self.trainer.iter + 1
        if next_iter % self._period != 0 and next_iter != self.trainer.max_iter:
            return

        latest_metric = self.trainer.storage.history(self._val_metric).latest()
        if latest_metric is None:
            return

        is_better = (
            self._best_metric is None or
            (self._mode == "min" and latest_metric < self._best_metric) or
            (self._mode == "max" and latest_metric > self._best_metric)
        )

        if is_better:
            self._best_metric = latest_metric
            save_name = f"{self._file_prefix}"
            self._checkpointer.save(save_name)
            print(f"[BestCheckpointer] Saved new best checkpoint to '{save_name}.pth' "
                  f"(val_loss: {latest_metric:.4f})")

In [ ]:
import torch
import time
from detectron2.engine.hooks import HookBase

class LossEvalHook(HookBase):
    def __init__(self, eval_period, model, data_loader):
        self._model = model
        self._data_loader = data_loader
        self._eval_period = eval_period

    def _do_loss_eval(self):
        total = 0.0
        count = 0
        num_devices = torch.distributed.get_world_size() if torch.distributed.is_initialized() else 1

        # self._model.eval()
        with torch.no_grad():
            for idx, inputs in enumerate(self._data_loader):
                if "instances" not in inputs[0]:  # ← inputs[0] に 'instances' がないと loss が出せない
                    print(f"[LossEvalHook] Warning: 'instances' not in inputs[{idx}]")
                    continue
                try:
                    loss_dict = self._model(inputs)
                    print(f"[LossEvalHook] loss_dict at idx={idx}:", loss_dict)
                    losses = sum(loss_dict.values())
                    total += losses.item()
                    count += 1
                except Exception as e:
                    print(f"[LossEvalHook] Error at idx={idx}: {e}")
        self._model.train()
        mean_loss = total / count if count > 0 else float("nan")
        print(f"[LossEvalHook] Validation Loss: {mean_loss:.4f}, Count: {count}")
        self.trainer.storage.put_scalar("val_loss", mean_loss)
        return mean_loss

    def after_step(self):
        next_iter = self.trainer.iter + 1
        if next_iter % self._eval_period == 0 and next_iter != self.trainer.max_iter:
            self._do_loss_eval()

    def after_train(self):
        self._do_loss_eval()

In [ ]:
from detectron2.utils.events import EventStorage
from detectron2.data import build_detection_test_loader, DatasetMapper

# Set is_train=True in the mapper so that 'instances' are included
val_loader = build_detection_test_loader(
    cfg,
    cfg.DATASETS.TEST[0],
    mapper=DatasetMapper(cfg, is_train=True)
)

trainer = MyTrainer(cfg)

# Register custom hooks
trainer.register_hooks([
    LossEvalHook(cfg.TEST.EVAL_PERIOD, trainer.model, val_loader),
    BestCheckpointer(cfg.TEST.EVAL_PERIOD, trainer.checkpointer, "val_loss", mode="min", file_prefix="best_model")
])

In [ ]:
trainer.register_hooks([
    LossEvalHook(cfg.TEST.EVAL_PERIOD, trainer.model, val_loader),
    BestCheckpointer(cfg.TEST.EVAL_PERIOD, trainer.checkpointer, "val_loss", mode="min", file_prefix="best_model0804")
])

In [ ]:
from detectron2.config import get_cfg
cfg = get_cfg()
print(cfg.INPUT)

In [ ]:
trainer.train()

In [ ]:
!jupyter nbconvert --to markdown --stdout keypoint11.ipynb > notebook_log0805.txt

In [ ]:
!cp -r runs runs_backup_$(date +%Y%m%d_%H%M%S)

In [ ]:
!tensorboard --logdir ./output_ddh0804/

In [ ]:
!find ./output_ddh0804/ -name "events.out.tfevents.*"

In [ ]:
!tensorboard --logdir .output_ddh0804/your_experiment_name/

In [ ]:
from detectron2.config.config import CfgNode as CN

In [ ]:
cfg.DATALOADER.NUM_WORKERS = 0
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x.yaml")

cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.001  # pick a good LR
cfg.SOLVER.MAX_ITER = 20000  # 300 iterations seems good enough for this toy dataset; you may need to train longer for a practical dataset
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128   #128   # faster, and good enough for this toy dataset (default: 512)
cfg.SOLVER.CHECKPOINT_PERIOD = 1000
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # DDHxp1
cfg.MODEL.RETINANET.NUM_CLASSES = 1
cfg.MODEL.ROI_KEYPOINT_HEAD.NUM_KEYPOINTS = 10
cfg.TEST.KEYPOINT_OKS_SIGMAS = np.ones((10, 1), dtype=float).tolist()
cfg.TEST.EVAL_PERIOD = 1000
from datetime import datetime

# Make the output directory unique by timestamp
cfg.OUTPUT_DIR = f"./output_ddh/run_{datetime.now().strftime('%Y%m%d_%H%M%S')}"

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

trainer = MyTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

In [ ]:
import json
import matplotlib.pyplot as plt

experiment_folder = 'output'

def load_json_arr(json_path):
    lines = []
    with open(json_path, 'r') as f:
        for line in f:
            lines.append(json.loads(line))
    return lines

experiment_metrics = load_json_arr("path/to/your/metrics_json")  # Set the path to Detectron2 training metrics JSON (if available)

In [ ]:
plt.rcParams['figure.figsize'] = [15, 8]
plt.plot(
    [x['iteration'] for x in experiment_metrics if 'validation_loss' in x], 
    [x['validation_loss'] for x in experiment_metrics if 'validation_loss' in x], color='blue')
plt.plot(
    [x['iteration'] for x in experiment_metrics if 'validation_loss' in x], 
    [x['loss_keypoint'] for x in experiment_metrics if 'validation_loss' in x], color='orange')

plt.legend(['validation_loss', 'loss_keypoint'], loc='upper left')

In [ ]:
print(experiment_metrics.keys())